In [4]:
from vtk_revise import read_vtk,write_vtk,to_polydata,write_vertices,remove_field
import numpy as np
from tqdm.notebook import tqdm
import trimesh
from queue import PriorityQueue

In [5]:
lh_white_path = "./surf/lh.CortexODE.white.vtk"
lh_pial_path = "./surf/lh.CortexODE.pial.vtk"
rh_white_path = "./surf/rh.CortexODE.white.vtk"
rh_pial_path = "./surf/rh.CortexODE.pial.vtk"

In [7]:
lh_white_path = './AllCortexData/GS_CortexODE/SUBJ_001_MR_BL/surf/lh.CortexODE.white.vtk'
lh_white = read_vtk(lh_white_path)

In [8]:
thick = lh_white['thickness']
print(max(thick))
print(min(thick))

7.189536822
0.034291269928


In [6]:
lh_white = read_vtk(lh_white_path)
lh_pial = read_vtk(lh_pial_path)

rh_white = read_vtk(rh_white_path)
rh_pial = read_vtk(rh_pial_path)

FileNotFoundError: File (c:\Users\pari0\OneDrive - 고려대학교\바탕 화면\newCortex\surf\lh.CortexODE.white.vtk) not found

In [12]:
lh_white_vertices = lh_white['vertices']
lh_white_faces = lh_white['faces'][:,1:]

rh_white_vertices = rh_white['vertices']
rh_white_faces = rh_white['faces'][:,1:]

lh_pial_vertices = lh_pial['vertices']
lh_pial_faces = lh_pial['faces'][:,1:]

rh_pial_vertices = rh_pial['vertices']
rh_pial_faces = rh_pial['faces'][:,1:]


In [13]:
lh_white_mesh = trimesh.Trimesh(vertices=lh_white_vertices, faces=lh_white_faces)
rh_white_mesh = trimesh.Trimesh(vertices=rh_white_vertices, faces=rh_white_faces)
# Get the vertex normals
lh_white_vertex_normals = lh_white_mesh.vertex_normals
rh_white_vertex_normals = rh_white_mesh.vertex_normals


print(lh_white_vertex_normals)

[[ 0.20408021 -0.1778264  -0.96266767]
 [-0.43874678 -0.14048142 -0.88756196]
 [-0.06856184 -0.28265054 -0.95676954]
 ...
 [ 0.1943248   0.37795157  0.9052019 ]
 [-0.01420271  0.62883832  0.77740636]
 [ 0.04641489  0.54770614  0.83538233]]


In [14]:
lh_pial_mesh = trimesh.Trimesh(vertices=lh_pial_vertices, faces=lh_pial_faces)
rh_pial_mesh = trimesh.Trimesh(vertices=rh_pial_vertices, faces=rh_pial_faces)

lh_pial_intersector = trimesh.ray.ray_triangle.RayMeshIntersector(lh_pial_mesh)
rh_pial_intersector = trimesh.ray.ray_triangle.RayMeshIntersector(rh_pial_mesh)



### one side of brain 

In [7]:
def get_distance_white2pial(subject_name,white_vertices,white_vtx_normal,pial_mesh_intersector):
    '''
    subject_name => (String) : name of this subject. ex ) 'patient_1_lh_brain'
    white_vertices => (numpy array) : white matter vertex position
    white_vtx_normal => (numpy array) : white matter vertext normals
    pial_mesh_intersector => (intersector by trimesh) : pial surface intersector 
    '''
    
    not_intersect_num = 0 
    
    result = []
    
    for i,(ray_origins,ray_directions) in tqdm(enumerate(zip(white_vertices,white_vtx_normal)),total=len(white_vertices)) : 
        
        ray_origins = ray_origins.reshape(1,3)
        ray_directions = ray_directions.reshape(1,3)
        # Intersect rays with mesh
        locations = pial_mesh_intersector.intersects_location(ray_origins, ray_directions)
        
        intersect_positions = locations[0]
        num_faces_intersect = len(locations[1])
        intersect_faces_idx = locations[2]
        pq = PriorityQueue()
        
        if num_faces_intersect == 0 : 
            result.append({'id' : subject_name,
                    'distance' : None,
                    'origin_position' : ray_origins,
                    'origin_index' : i,
                    'ray_direction' : ray_directions,
                    'intersect_position' : None,
                    'intersect_face_index' : None})
            not_intersect_num += 1
            
        else : 
            for j,intersect_pos in enumerate(intersect_positions) : 
                
                dist = np.sqrt(np.sum((ray_origins - intersect_pos)**2))
                pq.put((dist,
                        {'id' : subject_name,
                        'distance' : np.round(dist,4),
                        'origin_position' : ray_origins,
                        'origin_index' : i,
                        'ray_direction' : ray_directions,
                        'intersect_position' : intersect_pos,
                        'intersect_face_index' : intersect_faces_idx[j]}
                        ))
            small_one = pq.get()
            result.append(small_one[1])
    print(f"Vertex num : {len(white_vertices)}")
    print(f"Not intersect num : {not_intersect_num}")
    return result 


subject_name = 'lh_brain'
result = get_distance_white2pial(subject_name,
                                 lh_white_vertices,
                                 lh_white_vertex_normals,
                                 lh_pial_intersector)

print(result)


0it [00:00, ?it/s]

### total brain

In [15]:
combined_white_vertices =  np.concatenate((lh_white_vertices, rh_white_vertices), axis=0)
combined_white_vertices_norm =  np.concatenate((lh_white_vertex_normals, rh_white_vertex_normals), axis=0)

combined_pial_mesh = trimesh.util.concatenate(lh_pial_mesh, rh_pial_mesh)
combined_pial_intersector = trimesh.ray.ray_triangle.RayMeshIntersector(combined_pial_mesh)

In [ ]:
def get_distance_white2pial(subject_name,white_vertices,white_vtx_normal,pial_mesh_intersector):
    '''
    subject_name => (String) : name of this subject. ex ) 'patient_1_lh_brain'
    white_vertices => (numpy array) : white matter vertex position
    white_vtx_normal => (numpy array) : white matter vertext normals
    pial_mesh_intersector => (intersector by trimesh) : pial surface intersector 
    '''
    
    not_intersect_num = 0 
    
    result = []
    
    for i,(ray_origins,ray_directions) in tqdm(enumerate(zip(white_vertices,white_vtx_normal)),total=len(white_vertices)) : 
        
        ray_origins = ray_origins.reshape(1,3)
        ray_directions = ray_directions.reshape(1,3)
        # Intersect rays with mesh
        locations = pial_mesh_intersector.intersects_location(ray_origins, ray_directions)
        
        intersect_positions = locations[0]
        num_faces_intersect = len(locations[1])
        intersect_faces_idx = locations[2]
        pq = PriorityQueue()
        
        if num_faces_intersect == 0 : 
            result.append({'id' : subject_name,
                    'distance' : None,
                    'origin_position' : ray_origins,
                    'origin_index' : i,
                    'ray_direction' : ray_directions,
                    'intersect_position' : None,
                    'intersect_face_index' : None})
            not_intersect_num += 1
            
        else : 
            for j,intersect_pos in enumerate(intersect_positions) : 
                
                dist = np.sqrt(np.sum((ray_origins - intersect_pos)**2))
                pq.put((dist,
                        {'id' : subject_name,
                        'distance' : np.round(dist,4),
                        'origin_position' : ray_origins,
                        'origin_index' : i,
                        'ray_direction' : ray_directions,
                        'intersect_position' : intersect_pos,
                        'intersect_face_index' : intersect_faces_idx[j]}
                        ))
            small_one = pq.get()
            result.append(small_one[1])
    print(f"Vertex num : {len(white_vertices)}")
    print(f"Not intersect num : {not_intersect_num}")
    return result 


subject_name = 'all_brain'
result = get_distance_white2pial(subject_name,
                                 combined_white_vertices,
                                 combined_white_vertices_norm,
                                 combined_pial_intersector)

print(result)


In [2]:
# To load the list back:
import pickle
with open('lh_brain.pkl', 'rb') as f:
    loaded_list = pickle.load(f)

print(len(loaded_list))

142422


In [17]:
import trimesh
import numpy as np

# Sample vertices and faces
vertices = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

faces = np.array([
    [0, 1, 2],
    [0, 1, 3],
    [0, 2, 3],
    [1, 2, 3]
])

# Create a trimesh object
mesh = trimesh.Trimesh(vertices=lh_white_vertices, faces=lh_white_faces)

# Calculate vertex normals
vertex_normals = mesh.vertex_normals

print(vertex_normals)


[[ 0.20408021 -0.1778264  -0.96266767]
 [-0.43874678 -0.14048142 -0.88756196]
 [-0.06856184 -0.28265054 -0.95676954]
 ...
 [ 0.1943248   0.37795157  0.9052019 ]
 [-0.01420271  0.62883832  0.77740636]
 [ 0.04641489  0.54770614  0.83538233]]


In [16]:
import numpy as np

def compute_vertex_normals(vertices, faces):
    vertex_normals = np.zeros_like(vertices, dtype=np.float64)  

    for face in faces:
        v1, v2, v3 = vertices[face].astype(np.float64)  
        e1 = v2 - v1
        e2 = v3 - v1
        face_normal = np.cross(e1, e2)
        face_normal /= np.linalg.norm(face_normal)
        
        
        for vertex in face:
            vertex_normals[vertex] += face_normal

    
    for i in range(len(vertex_normals)):
        vertex_normals[i] /= np.linalg.norm(vertex_normals[i])

    return vertex_normals

vertices = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

faces = np.array([
    [0, 1, 2],
    [0, 1, 3],
    [0, 2, 3],
    [1, 2, 3]
])

print(compute_vertex_normals(lh_white_vertices, lh_white_faces))


[[ 0.18497026 -0.19298697 -0.96360886]
 [-0.41961876 -0.14318219 -0.89633641]
 [-0.09402466 -0.25659499 -0.9619347 ]
 ...
 [ 0.18993384  0.379502    0.90548516]
 [-0.02300177  0.63046376  0.7758778 ]
 [ 0.04777164  0.54589086  0.8364933 ]]


In [26]:
print(lh_white)
print('')
print(lh_white['new_thickness'])

{'vertices': array([[-33.0875 ,   5.35956, -30.0661 ],
       [-33.6076 ,   5.37578, -29.9285 ],
       [-33.2168 ,   4.80791, -29.9696 ],
       ...,
       [ -8.10695, -26.3459 ,  80.0389 ],
       [ -9.7135 , -25.4772 ,  79.6402 ],
       [ -8.80204, -25.6295 ,  79.7029 ]], dtype=float32), 'faces': array([[     3,      2,      1,      0],
       [     3,      0,      3,      2],
       [     3,      6,      5,      4],
       ...,
       [     3, 142334, 142420, 142336],
       [     3, 142336, 142420, 142421],
       [     3, 142336, 142421, 142331]], dtype=int64), 'thickness': array([3.32757632, 3.66757862, 3.19666151, ..., 3.7112704 , 3.66817025,
       3.49256995]), 'aparc': array([ 9.,  9.,  9., ..., 24., 24., 24.]), 'new_thickness': array(['3.18507367\r', '4.61491413\r', '3.35244491\r', ...,
       '3.46846138\r', '3.63713319\r', '3.51621519\r'], dtype='<U13')}

['3.18507367\r' '4.61491413\r' '3.35244491\r' ... '3.46846138\r'
 '3.63713319\r' '3.51621519\r']


In [21]:
hello = []

for i in range(100):
    hello.append(i)
    
    
a = hello[:10]
b = hello[10:]

print(len(a))
print(len(b))

10
90


In [22]:
write_vtk(lh_white,'hello.vtk')

In [4]:
import numpy as np


a = np.zeros(3)
b = np.array([1,2,2])

c = np.sqrt(np.sum((a-b)**2))

print(c)

3.0


In [31]:
from vtk import read_vtk,write_vtk,to_polydata,write_vertices,remove_field

lh_white = read_vtk('lh_white_not_combined.vtk')

print(lh_white)

{'vertices': array([[-33.0875 ,   5.35956, -30.0661 ],
       [-33.6076 ,   5.37578, -29.9285 ],
       [-33.2168 ,   4.80791, -29.9696 ],
       ...,
       [ -8.10695, -26.3459 ,  80.0389 ],
       [ -9.7135 , -25.4772 ,  79.6402 ],
       [ -8.80204, -25.6295 ,  79.7029 ]], dtype=float32), 'faces': array([[     3,      2,      1,      0],
       [     3,      0,      3,      2],
       [     3,      6,      5,      4],
       ...,
       [     3, 142334, 142420, 142336],
       [     3, 142336, 142420, 142421],
       [     3, 142336, 142421, 142331]], dtype=int64), 'thickness': array([3.32757632, 3.66757862, 3.19666151, ..., 3.7112704 , 3.66817025,
       3.49256995]), 'aparc': array([ 9.,  9.,  9., ..., 24., 24., 24.]), 'new_thickness': array([3.18507367, 4.61491413, 3.35244491, ..., 3.46846138, 3.63713319,
       3.51621519])}


In [28]:
def to_float(value):
    try:
        return float(value.replace('\r', ''))
    except ValueError:
        return np.nan

In [ ]:
import numpy as np

from vtk import read_vtk,write_vtk,to_polydata,write_vertices,remove_field
rh_white = read_vtk('rh_white_not_combined.vtk')

print(rh_white)
thickness = rh_white['new_thickness']
rh_white['new_thickness'] = np.array([to_float(x) for x in thickness])


write_vtk(rh_white,'rh_white_not_combined_revise.vtk')

print(rh_white)



In [35]:
rh_white = read_vtk('rh_white_not_combined.vtk')

print(rh_white)

{'vertices': array([[ 40.02   ,  -1.60078, -30.0925 ],
       [ 39.6111 ,  -1.93742, -29.7247 ],
       [ 40.1154 ,  -2.23625, -29.8329 ],
       ...,
       [  8.01175, -23.9332 ,  81.045  ],
       [  8.51985, -24.1705 ,  80.8635 ],
       [  8.07586, -23.4677 ,  80.8645 ]], dtype=float32), 'faces': array([[     3,      2,      1,      0],
       [     3,      0,      3,      2],
       [     3,      6,      5,      4],
       ...,
       [     3, 140420, 140428, 140413],
       [     3, 140429, 140423, 140418],
       [     3, 140423, 140429, 140421]], dtype=int64), 'thickness': array([3.9675797 , 4.42450906, 4.27335525, ..., 2.78195273, 2.99477607,
       2.69913931]), 'aparc': array([ 9.,  9.,  9., ..., 24., 24., 24.]), 'new_thickness': array([3.58736357, 4.16528196, 4.15888569, ..., 2.74520761, 2.96018984,
       2.61897407])}


In [28]:
import vtk

# Create two points
p1 = [1, 2, 3]
p2 = [4, 5, 6]

# Create a vtkPoints object and store the points in it
points = vtk.vtkPoints()
points.InsertNextPoint(p1)
points.InsertNextPoint(p2)

# Create a line between the two points
line = vtk.vtkLine()
line.GetPointIds().SetId(0, 0)  # the second 0 is the index of the first point in points
line.GetPointIds().SetId(1, 1)  # the second 1 is the index of the second point in points

# Create a cell array to store the line and insert the line into it
lines = vtk.vtkCellArray()
lines.InsertNextCell(line)

# Create a polydata to store everything
linesPolyData = vtk.vtkPolyData()
linesPolyData.SetPoints(points)
linesPolyData.SetLines(lines)

# Write the polydata to a file
writer = vtk.vtkPolyDataWriter()
writer.SetFileName("line.vtk")
writer.SetInputData(linesPolyData)
writer.Write()


AttributeError: module 'vtk' has no attribute 'vtkPoints'

In [5]:
import pickle
import numpy as np

# Open the file in binary read mode
with open('lh_brain.pkl', 'rb') as file:
    # Load the object from the file
    data = pickle.load(file)

# Now, 'data' contains the deserialized Python object
# print(data)


In [6]:
print(data[0])
origin_pos = np.squeeze(data[0]['origin_position']).astype(np.float64)
pial_pos = np.squeeze(data[0]['intersect_position']).astype(np.float64)


print(origin_pos,pial_pos)

{'id': 'lh_brain', 'distance': 3.1851, 'origin_position': array([[-33.0875 ,   5.35956, -30.0661 ]], dtype=float32), 'origin_index': 0, 'ray_direction': array([[ 0.20408021, -0.1778264 , -0.96266767]]), 'intersect_position': array([-32.43749102,   4.79316982, -33.13226661]), 'intersect_face_index': 1}
[-33.08750153   5.35956001 -30.06609917] [-32.43749102   4.79316982 -33.13226661]


In [7]:
import vtk


# Create a vtkPoints object and store the points in it
points = vtk.vtkPoints()
points.InsertNextPoint(origin_pos)
points.InsertNextPoint(pial_pos)

# Create a line between the two points
line = vtk.vtkLine()
line.GetPointIds().SetId(0, 0)  # the second 0 is the index of the first point in points
line.GetPointIds().SetId(1, 1)  # the second 1 is the index of the second point in points

# Create a cell array to store the line and insert the line into it
lines = vtk.vtkCellArray()
lines.InsertNextCell(line)

# Create a scalar array to store color values (assuming a single line here)
colors = vtk.vtkDoubleArray()
colors.SetName("Colors")  # Name the scalar
colors.InsertNextValue(1.0)  # Assuming a scalar value of 1.0 for this line

# Create a polydata to store everything
linesPolyData = vtk.vtkPolyData()
linesPolyData.SetPoints(points)
linesPolyData.SetLines(lines)
linesPolyData.GetCellData().SetScalars(colors)

# Write the polydata to a file
writer = vtk.vtkPolyDataWriter()
writer.SetFileName("line.vtk")
writer.SetInputData(linesPolyData)
writer.Write()



1